# Support Vector Machines

A Support Vector Machine commonly reffered to as an SVM is a powerful and very versatile machine learning model. SVM are one of the most popular Supervised Learning algorithms, they are capable of linear and non linear classification, regression and even outlier detection. Support Vector Machines are typically well suited for classification of complex small or medium sized datasets.

The goal of SVM is to create the best line or decision boundary that can be used to segregate n-dimensional space into classes so that we can easily put the new datapoint in the correct category in the future. This best decision boundary is called the hyperplane. SVM chooses extreme points/vectors (that are instances) that help create the hyperplane, these are called support vectors, hence the name of the algorithm. This process will be discussed in more detail later.

This chapter will be divided into the following sections: -

1. Linear SVM Classification
2. Nonlinear SVM Classification
3. SVM Regression
4. Working of SVM

## 1. Linear SVM Classification

<center>
<br>
<img src="https://www.researchgate.net/publication/304611323/figure/fig8/AS:668377215406089@1536364954428/Classification-of-data-by-support-vector-machine-SVM.png" height="400">
<br>
</center>

The fundamental idea behind SVM can best be explained by using the above image as an example. The two classes can easily be seperated by a straight line (i.e they are linearly seperable). 

The decision boundaries are marked by the dash lines. As stated earlier the support vectors are the ones that are used to make the hyperplanes. The instances are classified according to which side of the hyperplane they lie on. You can think of an SVM classifier as fitting the widest possible street (represented by parallel dashed lines) between the classes. This is called large margin classification. It should be noted that addeing more training instances off the street will not affect the decision boundary at all. Ot is fully determined/supported by the instances located on the edge of the street. These instances are called the support vectors.

<br>
<center>
<img src="https://www.oreilly.com/api/v2/epubs/9781787125933/files/graphics/B07030_03_09.jpg" height="350">
</center>
<br>

It must also be rememebered that SVMs are sensitive to feature scales, after feature scaling the decision boundary looks a lot better, i.e. the margin is a lot bigger.

<br>
<center>
<img src="https://miro.medium.com/max/1332/1*mKH7ePxH9xJ2Avsess9nzA.png" height="350">
</center>
<br>


### 1.1 Soft Margin Classification

We cannot always make the model in such a way that all the instances are off the street (street referring to the area between the hyperplanes). If we try to impose this then it is called hard margin classification. 

The two main issues with hard margin classification are that: -
1. It only works with linearly seperable data 
2. It is highly susceptible to outliers. 

For example if one of the instances of negative classes is in the group of the positive class instances when we plot, then it becomes impossible to seperate them in such a way to have hard margin i.e. no instances on the street. Another problem is that if we do make the decision boundary taking the outlier into account then the model will overfit and will have a hard time generalising.

To avoid these issues we use a more flexible model. The objective is to find a model that keeps the street as wide as possible and limit the margin violations (i.e. instances that end up in the middle of the street or even on the wrong side). This is called soft margin classification.

When creating a SVM model using sklearn, we can specify a number of hyperparameters. C is one of those hyperparameters. If we set it to a low value, then we end up with the model that will have a wide street but a few instances will be on the street. IF we set it to a higher value the street will be a lot narrower and we will however have fewer instances on the street. The prior model would be have more margin violations but will probably generalise better. Our goal is to find a good balance between these two. If the SVM model is overfitting, then you can regularise it by reducing the value of C.

Next lets implement Linear SVM Model and train it on the iris dataset. We will load the dataset, scale the feature and then train a Linear SVM Model (using LinearSVC class with C=1 and the hinge loss function) to detect Iris Virginica flowers.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris['data'][:, (2, 3)]  # to get petal length and petal width
y = (iris['target'] == 2).astype(float)

svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_svc', LinearSVC(C=1, loss="hinge"))
])

svm_clf.fit(X, y)

svm_clf.predict([[5.5, 1.7]])

array([1.])

Unlike Logistic Regression classes, SVM classifiers do not put probabilities of each class. Instead of using the LinearSVC class, we could also the SVC class with a linear kernel. To do this we would write SVC(kernel="linear", C=1). Or we could use the SGDClassifier class, with SGDClassifier(loss="hinge", alpha=1/(m*C)). This applies regular Stochastic Gradient Descent to train a linear SVM classifier. It does not converge as fast as the linear SVC class, but it can be used to handle online classification tasks or huge datasets that do not fit in memory (out-of-core training).

The LinearSVC class regularises the bias term, so you should center your training set first by subtracting its mean. This is automatic if you train the data using the standard scaler. Also make sure the loss hyperparameter is set to "hinge" as it is not the default value. For better performance, you should set the dual hyperparameter to False unless there are more features than training instances.


## 2. Nonlinear SVM Classification

Linear SVM Classifiers are efficient and work very well, however a vast majority of datasets are not even close to being linearly seperable. One approach to handling non linear datasets, is to add more features, such as polynomial features, in some cases this can result in a linearly seperable dataset. For example a dataset with just one feature x1 may not be linearly seperable but if another feature is added, the resulting 2D dataset is perfectly linearly seperable.

<br>
<center>
<img src="https://miro.medium.com/max/1400/1*NwhqamsvzBkUlYwSAubv5g.png" height="300">
</center>
<br>

To implement this using sklearn, we will create a Pipeline containing a PolynomialFeatures transformer, followed by a Standard Scaler and a Linear SVC. We can test this on the moons dataset: a toy dataset for binary classification in which data points are shaped as two interleaving half circles. You can generate this dataset using the make_moons() function.

We will now implement this in python.

In [2]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import matplotlib.pyplot as plt

X, y = make_moons(n_samples=100, noise=0.15)
poly_svm_clf = Pipeline([
    ('poly_features', PolynomialFeatures(degree=3)),
    ('scaler', StandardScaler()),
    ('svm_clf', LinearSVC(C=10, loss="hinge", max_iter=10000))  # failed to converge, use a value less than 1k
])

poly_svm_clf.fit(X, y)

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge', max_iter=10000))])

<br>
<center>
<img src="https://img2018.cnblogs.com/blog/1012590/201903/1012590-20190331183702438-196976647.png" height="400">
</center>
<br>

### 2.1 Polynomial Kernel

Adding polynomial features is very useful as because of this we can get our models to fit and perform well on non-linear data. Adding these features make models work great and it is also very simple to implement. However at a low polynomial degree, this method cannot deal with very complex datasets, and at large degrees it becomes very slow because of the combinatorial explosion, creating a large number of features, making the model very slow.

There is however a trick to get around this which is called the kernel trick. The kernel trick makes it possible to get the same result as if you had added many polynomial features without actually adding them. So there is no combinatorial explosion and the model does not become slow. Trick can be impleemented using the SVC class. We implement it in the following manner.

In [3]:
from sklearn.svm import SVC

poly_kernel_svc = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(kernel='poly', degree=3, coef0=1, C=5))
])

poly_kernel_svc.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

This code trains an SVM classifier using a third degree polynomial kernel. If your model is overfitting, then reduce the degree, if underfitting you can increase it. The hyperparameter coef0 controls how much the model is influenced by high-degree polynomials vs low-degree polynomials.

A common approach to find the right hyperparameter value is to use grid search. It is often faster to do a very coarse grid search, then user a finer grid search around the best values found. Hvaing a good sense of what each hyperparameter does can also help search the right part of the hyperparameter space.


### 2.2 Similarity Features

Another technique to tackle nonlinear problems is to add features computed using a **similarity function**, which measures how much each instance resembles a particular landmark. For example taking the 1D dataset and adding two landmarks to it at $x_{1} = -2$ and $x_{2} = 1$. Next we will define a similarity function to be the Gaussian Radial Basis Function (RBF) with $\gamma = 0.3$

Gaussian RBF:

$\phi_{\gamma}(x, \ell) = \exp(-{\gamma} || x - \ell || ^2)$

This is a bell shaped function varying from 0 (very far away from landmark) to 1 (at the landmark). Now we can compute the new features. For example taking instance $x_{1} = -1$, it is located at a distance of 1 from the first landmark and 2 from the second landmark. Therefore its new features are $x_{2} = exp(-0.3 x 1^2) \approx 0.74$ and $x_{3} = exp(-0.3 x 2^2) \approx 0.30$. The right plot shows the transformed dataset without the original features. Now it is observed that it is linearly seperable.


<br>
<center>
<img src="https://miro.medium.com/max/1306/1*_9go20Mzy5ECwLi-syNrYw.png" height="400">
</center>
<br>

The main question that arises is how do we choose the landmarks. The simplest approach is to make a landmark at the position of every instance in the dataset. Doing so will create many dimensions and thus the changes of our instances being linearly seperable increases. The problem that arises with this is that if your dataset has n feature and m instances before we perform the process after we do so, there will be m features and m instances, assuming you drop the original features. This will make it linearly seperable but can cause problems if the dataset is very large, as it will after the transformation have an equally large number of features.


### 2.3 Gaussian RBF Kernel

Just like the polynomial features method, the similarity features method can be used with any machine learning algorithm, it could be computationally heavy because of the large number of features that would be created if you used a large dataset. Once again we use the kernel trick, that makes it possible to obtain a similar result as if you added many similarity features. An implementation of the SVC Class with the Gaussian RBF kernel would be as follows:

In [4]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])

rbf_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

Model is represented at the bootom. The other plots are other models that were trained on other gamma $\gamma$ and C values. Increasing the value of gamma makes the bell shaped curve narrower as a result the decision boundary becomes more irregular and tends to very closelt go around the instances. This is because as you increase the value of gamma, each instances range of influence becomes smaller, when you reduce gamma the influence of individual instances becomes larger and the bell curve becomes smoother. If your model is overfitting, you should reduce the alue of gamma, i.e. increase the influence of individual instances, which will make the bell curve smoother and it wont overfit.

<br>
<center>
<img src="https://img2018.cnblogs.com/blog/1012590/201903/1012590-20190331223623432-1521850923.png" height="400">
</center>
<br>

Other kernels are also there but they are rarely used. Some kernels are specialised ofr specific data structures. String kernels are sometimes used when classifying text documents or DNA sequences (using the string subsequence kernel or kernels based on the Levenshtein distance).

There are multiple kernels to choose from : linear, rbf, sigmoid ... So you might be confused on which one to choose. You must always start off by trying to use the linear kernel, (NOTE: LinearSVC is much faster than SVC(kernel="linear")), this is especially true if your training set is very large or you have a large amount of features. if the training set is not too large, you can also try using Gaussian RBF Kernel, it works well in most cases. After this if you yet want to experiment you can use cross validation and grid search to find what suits your data. Some kernels work better for specific data structures as stated earlier.


### 2.4 Computational Complexity

The LinearSVC class is based on the liblinear library, it implements an optimised algorithm for Linear SVMs but it does not support the kernel trick, but it scales almost linearly with the number of training instances and the number of features. Its training time is roughly $O$ ($m$ x $n$)

The algorithm takes longer if you require a very high precision. This is controlled by the tolerance hyperparameter $\epsilon$ (called tol in sklearn). In most classification tasks, the default tolerance is fine.

The SVC class is based on the libsvm library, which implements an algorithm that supports the kernel trick unlike the LinearSVC class. The training time complexity is between $O$ ($m^2$ x $n$ ) and $O$ ($m^3$ x $n$).

Because of this time complexity, we know that it gets very slow as the number of training instances get large. This algorithm is perfect for handling complex small - medium sized datasets. It scales well with the number of fatures especially with sparse features (each instance has few non-zero features). In this case algorithm scakes roughly with the average number of nonzero features per instance.

<br>
<br>
<center>
<img src="https://img2018.cnblogs.com/blog/1012590/201903/1012590-20190331234705484-1218971349.png" height="250">
</center>
<br>


## 3. SVM Regression

The SVM algorithm is very versatile, not only can it be used for classification tasks, it can also be used for regression tasks. The goal however is opposite, while in classification we tried to make the street as large as possible and margin violations were the instances that were in the street, in regression we try to make the street as narrow as possible and fit in all the instances in the street, here margin violations are when the instances are not there in the street but are outside it.

The width of the street is controlled by a hyperparameter called $\epsilon$, the figure given below hows two linear svm regression models trained on some random linear data, one with  alarge margin and the other with a small margin. Adding more training instances within the margin does not afect the models performance that is the model is $\epsilon$-insensitive.


You can use sklearns LinearSVR class to perform linear SVM regression. Always remember to scale and center the data.

In [5]:
from sklearn.svm import LinearSVR

svm_reg = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_reg', LinearSVR(epsilon=1.5))
])

svm_reg.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_reg', LinearSVR(epsilon=1.5))])

To tackle nonlinear regression tasks, you can use a kernelised SVm model. The following diagram shows SVM regression on a random quadratic training set, we use a second degree polynomial kernel. There is a litte regularisation in the left plot (i.e. a much larger C value), and much more regularisation in the right plot (a small C value).

<br>
<center>
<img src="https://miro.medium.com/max/1400/1*1MvBY0uWo69Z2vfFF6hCNA.png" height="300">
</center>
<br>

Following we have an implementation that uses sklearn SVR class (supports kernel trick) to produce the model thats on the left.

In [6]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel='poly', degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y)

SVR(C=100, degree=2, kernel='poly')

The SVR class is the regression equivalent of the SVC class, and the LinearSVR class is the regression equivalent of the LinearSVC class. The LinearSVR class scales linearly with the size of the training set (like LinearSVC class), while the SVR class gets too slow when the training set grows large (like th SVC class).

SVMs can be also be used for outlier detection.

## 4. Working of SVM

We will now further explore how SVMs make predictions and how their training algorithms work starting with linear SVM classifiers.

First we will start by looking into notations. Earlier in regression we discussed the convention of putting all the model parameters in one vector $\theta$, including the bias term $\theta_{0}$ and the input feature weights $\theta_{1}$ to $\theta_{n}$, and adding a bias input $x_{0}= 1$ to all instances. Now weill use a convention that is more convenient and common when dealing with SVMs. The bias term will be called b, and the feature weights vector will be w. No bias feature will be added to the input feature will be added to the input feature vetors.

### 4.1 Decision Funtion and Predictions

The linear SV< classifier model predicts the class of a new instance x by computing the decision function: 

$w^Tx + b = w_{1}x_{1} + ... + w_{n}x_{n}$. 

If the result is positive, the predicted class $\hat{y}$ is the positive class (1), else it is the negative class (0).

$\hat{y} = 0 if w^Tx + b < 0 or 1 if w^Tx + b >= 0$

The decision fucntion is given below. THe decision boundary is the set of points where the decision fucntion is equal to 0, it is the intersection of two planes, which is a straight line.

Dashed lines represent the points where the decision fcuntion is equal to 1 or -1, they are parallel and at equal distance to the decision boundary, and they fprm a margin around it. Training a SVM classifier means finding the values of w and b that make this margin as wide as possible while avoiding margin violations (hard magin) or limiting them (soft margin).

### 4.2 Training Objective

If we consider the slope of a decision function: it is equal to the norm of the weight vector $||w||$. If we divide this slope by 2, the points where the decision function is equal to $\pm1$ are going to be twice as far away from the decision boundary. This means that dividing the slope by 2 will multiply the margin by 2. The smaller the weight vector $w$, the larger the margin.

<br>
<center>
<img src="https://miro.medium.com/max/1240/1*0HncfVU3eh_eU_CsTL9XPg.png" height="300">
</center>
<br>

So the goal is to minimise $|| w || $ to get a large margin. If we want to avoid any margin violations (hard margin), then we need the decision fucntion to be greater than 1 for all positive training instances and lower than -1 for negative training instances. If we define $t^{(i)} = -1$ for negative instances (if $y^{i} = 0$) and $t^{(i)} = 1$ for positive instances (if $y^{i} = 1$), then we can express this constraint as $t^{(i)}(w^Tx^{(i)} + b \ge 1)$ for all instances.

Equation for Hard margin linear SVM classifier objective: 

$\frac{minimize}{w, b} \frac{1}{2}w^Tw$

subject to:  $t^{(i)}(w^Tx^{(i)} + b) \ge 1$   for i = 1,2,.. m

To get the soft margin objective we need to introduce a slack variable $\xi^{(i)} \ge 0$ for each instance, $\xi^{(i)}$ measures how much the $i^{th}$ instance is allowed to violate the margin. We now have two conflicting objectives, make the slack variables as small as possible to reduce the margin violations and make $\frac{1}{2}w^Tw$ as small as possible to increase the margin. This is where the hyperparameter C comes in, it allows us to define the tradeoff between these two objectives. This gives us the constrained optmization problem.

Equation for Soft margin linear SVM classifier objective: 

$\frac{minimize}{w, b, \xi} \frac{1}{2}w^Tw + C \sum \limits_{i=1}^{m} \xi^{(i)}$

subject to:  $t^{(i)}(w^Tx^{(i)} + b) \ge 1 - \xi^{(i)}$ and $\xi^{(i)} \ge 0$  for i = 1,2,.. m


### 4.3 Quadratic Programming

The hard margin and spft margin problems are both convex quadratic optimization problems with linear constraints. Such problems are known as Quadratic Programming problems.

Minimise  $\frac{1}{2}p^THp + f^Tp$ 
p subject to: $Ap \le b$

where,
* $p$ is an $n_p$ dimensional vector where ($n_p$ = number of parameters)
* $H$ is an $n_p x n_p$ dimensional vector
* $f$ is an $n_p$ dimensional vector
* $A$ is an $n_c x n_p$ dimensional vector where ($n_c$ = number of constraints)
* $b$ is an $n_c$ dimensional vector

One way to train a hard margin linear SVM classifier is to use an off the shelf QP solver and pass it the various parameters. The resulting vector p will contain the bias term $b = p_{0}$ and the feature weights $w_{i} = p_{i}$ for i = 1,2, .. n. Similarly you can use a QP solver to solve the soft margin problem. To use the kernel trick however you will need to look at a different optimisation problem.

### 4.4 The Dual Problem

Given a constrained optimisation problem, known as the primal problem, it is possible to express a different but closely related problem, called its dual problem. The solution to the gual problem typically givea a lower bound to the solution of the primal problem but under some conditions it can have the same solution as the primal problem. Luckily the SVM problem meets these conditions (i.e the objective function is convex and thre inequality constraints are continuously differentiable and convex functions), so you can choose to solve the primal problem or the dual problem, both will have the same solution.

Dual Solution of the linear SVM objective

$\frac{minimise}{\alpha} \frac{1}{2} \sum \limits_{i=1}^{m} \sum \limits_{j=1}^{m} \alpha^{i} \alpha^{j} t^i t^j x^{(i)T} x^j - 
\sum\limits_{i=1}^{m} \alpha^{i}$

subject to $\alpha^{i} \ge 0$ for i = 1,2,..,m

Once you find the vector $\hat{\alpha}$ that minimises this equation (use QP solver), use the following equation to compute $\hat{w}$ and $\hat{b}$ that minimises the primal problem.  

$\hat{w} = \sum \limits_{i=1}^{m} \hat{\alpha}^{i}t^i x^i $

$\hat{b} = \frac{1}{n_s} \sum \limits_{i=1}^{m} (t^i - \hat{w}^Tx^i $

where $\hat{\alpha}^i > 0$

The dual problem is faster to solve than the primal one when the number of training instances is smaller than the number of fetures. More importantly the dual problem makes the kernel trick possible, while the primal does not. Now we will look into the kernel trick.

### 4.5 Kernelised SVMs

in the scenario where you want to apply a second degree polynomial transformation to a two dimensional training dataset, then train a linear SVM classifier on the transformed training set. The following equation shows the second degree polynomial mapping function $\phi$ that you want to apply.

The transformed vector is 3D instead of 2D. Now lets see what happens to a couple of 2D vectors, a and b if we apply this second degree polynomial mapping and then compute the dot product of the transformed vectors.

The dot product of the transformed vectors is equal to the square of the dot product of the original vectors: $\phi(a)^T\phi(b) = (a^Tb)^2$

The key insight is: if you apply the transformation \phi to all training instances then the dual problem will contain the dot product $\phi(x^i)^T\phi(x^j)$ but if the $\phi$ is the second degree polynomial transformation then you can replace the dot product of transformed vectors simply by $(x^{(i)T}x^{(j)})^2$. So you dont need to transform all the training instances, just replace the dot product by its square. The result will be strictly the same as if you had gone through the trouble of transforming the training set then fitting a linear SVM algorithm but this trick makes the whole process much more computationally efficient.

The function $K(a, b) = (a^Tb)^2$ is a second degree polynomial kernel. In machine learning, a kernel is a fucntion capable of computing the dot product $\phi(a)^T \phi(b)$, based only on the original vectors a and b wthout having to compute or even know about the transformation $\phi$. The following are some of the most widely used kernels: 

* Linear: $K(a, b) = a^Tb$
* Polynomial: $K(a, b) = (\gamma a^Tb + r)^d$
* Gaussian RBF: $K(a, b) = exp(-\gamma || a - b ||^2)$
* Sigmoid: $K(a, b) = tanh(\gamma a^Tb + r)$

There is a further process to convert this to the primal solution.


### 4.6 Online SVMs

This is the final section on SVMS and it is on Online SVM Classifiers (online means it learns incrementally, as new instances arrive).

For linear SVM classifiers, one method for implementing an online SVm classifier is to use Gradient Decent (e.g. using SGDClassifier) to minimise the cost function given beloew which is derived from the primal problem. Unfortunately Gradient Descent converges much slower than the methods based on QP.

Linear SVM classifier cost function:

$J(w,b) = \frac{1}{2}w^Tw + C\sum \limits_{i=1}^{m}\max(0, 1-t^{(i)}(w^Tx^{i} + b))$

The first sum in the cost function will push the model to have a small weight vector w, leading to a larger margin. The second sum computes the total of all margin violations. An instances margin violation is equal to 0 if it is located off the street and on the correct side or else it is proportional to the ditance to the correct side of the street. Minimising this term ensures the model makes the margin violations as small and as few as possible.

it is also possible to implement online kernelised SVMs, as described in papers "incremental and Decremental Support Vector Machine Learning" and "Fast Kernel Classifiers with Online and Active Learning". These kernelised SVMs are implemented in Matlab and C++. For large scale problems, you may want to use neural networks instead.


**Hinge Loss:** the function max(0, 1- t) is called the hinge loss function. It is equal to 0 when t %/ge% . Its derivative (slope) is equal to -1 if t < 1 and 0 if t > 1. It is not differentiable at t = 1, but just like for Lasso Regression, you can still use Gradient Descent using any subderivative at t=1 (any value between -1 and 0).

<br>
<img src="https://datamonje.com/wp-content/uploads/2022/01/Hinge-loss.png" height="350">
<br>


## 5. Exercises

1. Train a LinearSVC on a linearly seperable dataset. Then train an SVC and a SGDClassifier on the same dataset. See if you can get them to produce roughly the same model.
2. Train an SVM classifier on the MNIST dataset. Since the SVM classifiers are binary, you will need to use the one-vs-rest to classify for all ten digits. You will also have to tune the hyperparameters using small validation sets to speed up the process. What accuracy can you achieve? (sklearn dataset)
3. Train an SVM regressor on the California housing dataset (sklearn dataset)